# `reflect`

In [1]:
$go get github.com/davecgh/go-spew/spew

import "github.com/davecgh/go-spew/spew"

go: downloading github.com/davecgh/go-spew v1.1.1


In [2]:
import "reflect"
import "fmt"
import "strings"

In [10]:
// tiny formating objects
func f(s interface{}) string {
    d := fmt.Sprintf("%#+v", s)
    d = strings.Replace(d, "{", "{\n  ", -1) 
    d = strings.Replace(d, "}", "\n}\n", -1) 
    d = strings.Replace(d, ", ", ",\n  ", -1)  
    return d
}

Our test objects

In [3]:
type Test struct {
    Hi string `json:"hi,omitempty"`
    hi string 
}

In [6]:
h := Test{Hi:"Ola!", hi:"privit"}

```go
package main

import (
	"fmt"
	"reflect"
)

type Test struct {
	Hi string `json:"hi,omitempty"`
	hi string
}

func main() {
	h := Test{Hi: "Ola!", hi: "privit"}
	fmt.Println(reflect.ValueOf(h).FieldByName("hi").String())
}
```

In [10]:
reflect.ValueOf(h).FieldByName("hi").String()

ERROR: reflect: call of reflect.Value.Elem on zero Value

In [6]:
func (t *Test) SayHello(name string, age int) (ok bool) {
    return true
}

Let's play!

In [7]:
var t = Test{
    Hi: "!HOLA!",
    hi: "Say my name %s",
}

In [8]:
spew.Dump(t)

(struct { Hi string "json:\"hi,omitempty\""; 𒀸hi string }) {
 Hi: (string) (len=6) "!HOLA!",
 𒀸hi: (string) (len=14) "Say my name %s"
}


In [11]:
f(t)

struct {
   Hi string "json:\"hi,omitempty\""; 𒀸hi string 
}
{
  Hi:"!HOLA!",
  𒀸hi:"Say my name %s"
}


In [9]:
spew.Dump(reflect.TypeOf(t))

(*reflect.rtype)(0xc00058a780)(struct { Hi string "json:\"hi,omitempty\""; 𒀸hi string })


In [9]:
f(reflect.ValueOf(t))

struct {
   Hi string "json:\"hi,omitempty\""; 𒀸hi string 
}
{
  Hi:"!HOLA!",
  𒀸hi:"Say my name %s"
}


In [10]:
f(reflect.KindOf(t))

ERROR: repl.go:1:3: not a type: reflect.KindOf <*ast.SelectorExpr>

### `Type`

In [11]:
reflect.TypeOf(int64(1)).Kind()

int64

In [12]:
value := int64(1)

In [13]:
reflect.TypeOf(value).Name()

int64

In [14]:
reflect.String == reflect.TypeOf(value).Kind()

false

In [15]:
reflect.Int64 == reflect.TypeOf(value).Kind()

true

In [16]:
reflect.TypeOf(reflect.String).Kind() == reflect.TypeOf(reflect.Uint8).Kind()

true

### `Setting Value`

In [17]:
var x float64 = 1.1
s := reflect.ValueOf(&x)
_, _ = fmt.Printf("%s - %t", s.Type(), s.CanSet())

// s.SetFloat(2.3)

*float64 - false

In [18]:
// moving by pointer to value
v := s.Elem()
v.SetFloat(2.3)
_, _ = fmt.Printf("%s - %t", v.Type(), v.CanSet())
x

float64 - true

2.3

### `Structs`

In [19]:
t := Test{
    Hi: "!HOLA!",
    hi: "Say my name %s",
}
x := reflect.ValueOf(t)

In [20]:
f(t)

struct {
   Hi string "json:\"hi,omitempty\""; 𒀸hi string 
}
{
  Hi:"!HOLA!",
  𒀸hi:"Say my name %s"
}


In [21]:
x.NumField()

2

In [22]:
// filed by index
f(x.Field(1))

"Say my name %s"

In [23]:
// private field will fail
x.FieldByName("hi")

<invalid reflect.Value>

In [24]:
// filed by index
f(x.Field(0))

"!HOLA!"

In [25]:
// private field will fail
x.FieldByName("Hi")

!HOLA!

In [26]:
tf := reflect.TypeOf(t)
f(tf)

&reflect.rtype{
  size:0x20,
  ptrdata:0x18,
  hash:0xcc19137c,
  tflag:0x0,
  align:0x8,
  fieldAlign:0x8,
  kind:0x19,
  equal:(func(unsafe.Pointer,
  unsafe.Pointer) bool)(0x8d6780),
  gcdata:(*uint8)(0xc00020cc30),
  str:-190,
  ptrToThis:0
}


In [27]:
tf.NumField()

2

In [28]:
tf.Field(0)

{Hi  string json:"hi,omitempty" 0 [0] false}

In [29]:
tf.Field(1)

{𒀸hi  string  16 [1] false}

In [30]:
tf.FieldByName("hi")

{  <nil>  0 [] false} false

In [31]:
tf.FieldByName("Hi")

{Hi  string json:"hi,omitempty" 0 [0] false} true

In [32]:
v1 := tf.FieldByName("Hi")
f(v1)

// warning: expression returns 2 values, using only the first one: [reflect.StructField bool]


reflect.StructField{
  Name:"Hi",
  PkgPath:"",
  Type:(*reflect.rtype)(0x1634720),
  Tag:"json:\"hi,omitempty\"",
  Offset:0x0,
  Index:[]int{
  0
}
,
  Anonymous:false
}


In [33]:
tag := v1.Tag

In [34]:
tag.Get("json")

hi,omitempty

In [35]:
tag.Get("yaml")

In [36]:
v1,e1 := tag.Lookup("json")
fmt.Sprintf("[%+v] - %t", v1, e1)   

[hi,omitempty] - true

### `Functions`

In [37]:
x2 := reflect.TypeOf(t.SayHello)

In [38]:
x1 := reflect.TypeOf(t)
for i := 0; i < x1.NumField(); i++ {
    fmt.Printf("%s\n", f(x1.Field(i)))  
}

reflect.StructField{
  Name:"Hi",
  PkgPath:"",
  Type:(*reflect.rtype)(0x1634720),
  Tag:"json:\"hi,omitempty\"",
  Offset:0x0,
  Index:[]int{
  0
}
,
  Anonymous:false
}

reflect.StructField{
  Name:"𒀸hi",
  PkgPath:"",
  Type:(*reflect.rtype)(0x1634720),
  Tag:"",
  Offset:0x10,
  Index:[]int{
  1
}
,
  Anonymous:false
}



In [39]:
//Count inbound parameters
x2.NumIn()

2

In [40]:
//Count outbound parameters
x2.NumOut()

1

In [41]:
// Method
x2.String()

func(string, int) bool

In [42]:
x2.IsVariadic()

false

In [43]:
x2.PkgPath()

In [44]:
var out []string
for i := 0; i < x2.NumIn(); i++ {
    inV := x2.In(i)
    res := fmt.Sprintf("param[%d]  = kind: (%v) name: (%v)",i, inV.Kind(), inV.Name()) 
    out = append(out, res)
}

f(out)

[]string{
  "param[0]  = kind: (string) name: (string)",
  "param[1]  = kind: (int) name: (int)"
}


In [45]:
var out []string
for i := 0; i < x2.NumOut(); i++ {
    inV := x2.In(i)
    res := fmt.Sprintf("param[%d]  = kind: (%v) name: (%v)",i, inV.Kind(), inV.Name()) 
    out = append(out, res)
    fmt.Printf("%s", f(inV))
}

f(out)

&reflect.rtype{
  size:0x10,
  ptrdata:0x8,
  hash:0xe0ff5cb4,
  tflag:0x7,
  align:0x8,
  fieldAlign:0x8,
  kind:0x18,
  equal:(func(unsafe.Pointer,
  unsafe.Pointer) bool)(0x8169c0),
  gcdata:(*uint8)(0x18368b0),
  str:23238,
  ptrToThis:434144
}


[]string{
  "param[0]  = kind: (string) name: (string)"
}
